In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from scipy.spatial import Voronoi, voronoi_plot_2d

In [8]:
weekOneTracking = pd.read_csv("tracking_week_1.csv")
players = pd.read_csv("players.csv")

In [10]:
weekOneTracking = pd.merge(
    weekOneTracking,
    players[['nflId','position']],
    how = "left",
    left_on=['nflId'],
    right_on=['nflId']
)

In [11]:
test = weekOneTracking[:100000]

In [14]:
import numpy as np
import pandas as pd
from scipy.spatial import Voronoi

def voronoi_area(df):
    
    new_rows = []
    results = []

    # Adjust x-coordinates for WR, TE, RB positions and add new rows
    for _, row in df.iterrows():
        if row['position'] in ['WR', 'TE', 'RB']:
            new_row = {
                'x': row['x'] - 5,
                'y': row['y'],
                'position': '',
                'club': None,
                'nflId': 0
            }
            new_rows.append(new_row)

    # Combine original and new rows
    new_rows_df = pd.DataFrame(new_rows)
    filteredSamplePlayPoints = pd.concat([df, new_rows_df], ignore_index=True)

    # Field boundaries for clipping
    x_min, x_max = 0, 100
    y_min, y_max = 0, 53.3
    boundary_points = np.array([
        [x_min, y_min],
        [x_min, y_max],
        [x_max, y_min],
        [x_max, y_max]
    ])

    # Function to calculate the area of a polygon given its vertices
    def polygon_area(vertices):
        return 0.5 * np.abs(np.dot(vertices[:, 0], np.roll(vertices[:, 1], 1)) - 
                            np.dot(vertices[:, 1], np.roll(vertices[:, 0], 1)))

    # Group data by gameId, playId, and frameId
    grouped = filteredSamplePlayPoints.groupby(['gameId', 'playId', 'frameId'])

    # Process each group to calculate Voronoi areas for each player in each frame
    for (gameId, playId, frameId), group in grouped:
        points = group[['x', 'y']].to_numpy()
        all_points = np.vstack([points, boundary_points])
        vor = Voronoi(all_points)

        # Calculate Voronoi areas for each player in the group
        for i, region_index in enumerate(vor.point_region[:len(points)]):
            vertices = vor.regions[region_index]
            if -1 not in vertices:  # Only process bounded regions
                polygon = vor.vertices[vertices]
                polygon[:, 1] = np.clip(polygon[:, 1], y_min, y_max)
                area = polygon_area(polygon)
                
                # Retrieve player details
                nflId = group.iloc[i]['nflId']
                position = group.iloc[i]['position']

                # Store results for WR, TE, and RB positions
                if position in ['WR', 'TE', 'RB']:
                    results.append((gameId, playId, nflId, position, frameId, area))

    # Convert results to a DataFrame
    areas_df = pd.DataFrame(results, columns=['gameId', 'playId', 'nflId', 'position', 'frameId', 'area'])

    # Find the max area per player per play
    max_area_df = areas_df.groupby(['gameId', 'playId', 'nflId', 'position','frameId'], as_index=False).agg(max_area=('area', 'max'))

    return max_area_df



In [15]:
result = voronoi_area(test)
result.head(10)

,gameId,playId,nflId,position,frameId,max_area
0,2.022091e+09,64.0,42412.0,WR,1.0,371.737112
1,2.022091e+09,64.0,42412.0,WR,2.0,362.155190
2,2.022091e+09,64.0,42412.0,WR,3.0,327.221380
3,2.022091e+09,64.0,42412.0,WR,4.0,297.966339
4,2.022091e+09,64.0,42412.0,WR,5.0,280.196864
5,2.022091e+09,64.0,42412.0,WR,6.0,268.140432
6,2.022091e+09,64.0,42412.0,WR,7.0,260.023014
7,2.022091e+09,64.0,42412.0,WR,8.0,254.660210
8,2.022091e+09,64.0,42412.0,WR,9.0,250.921425
9,2.022091e+09,64.0,42412.0,WR,10.0,249.057472
